In [265]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from snowflake.connector.pandas_tools import write_pandas
from sqlalchemy import create_engine
import snowflake.connector

In [2]:
import snowflake.connector

In [5]:
## Set options to view all columns
pd.set_option('display.max_columns', None)

In [7]:
SNOWFLAKE_USER = 'kbharaj3'
SNOWFLAKE_PASSWORD = 'Snowfl@key0014'
SNOWFLAKE_ACCOUNT = 'qx25653.ca-central-1.aws'
SNOWFLAKE_WAREHOUSE = 'FOOTY_STORE'
SNOWFLAKE_DATABASE = 'GEGENSTATS'
SNOWFLAKE_SCHEMA = 'FBREF_TEAMSTATS'

In [26]:
conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
    warehouse=SNOWFLAKE_WAREHOUSE,
    database=SNOWFLAKE_DATABASE,
    schema=SNOWFLAKE_SCHEMA
)

In [27]:
cursor = conn.cursor()

In [220]:
cursor.execute('SELECT * FROM TEAMS')
team_rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
team_names = pd.DataFrame(team_rows, columns=column_names)

In [221]:
cursor.execute('SELECT * FROM TEAM_STANDARD_STATS')
standard_rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
team_standard = pd.DataFrame(standard_rows, columns=column_names)

In [222]:
cursor.execute('SELECT * FROM TEAM_ATTACKING_STATS')
attacking_rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
team_attacking = pd.DataFrame(attacking_rows, columns=column_names)

In [223]:
cursor.execute('SELECT * FROM TEAM_DEFENDING_STATS')
defending_rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
team_defending = pd.DataFrame(defending_rows, columns=column_names)

In [224]:
team_standard = team_standard.merge(team_names, on='TEAM_FBREF_ID', how='left')
team_defending = team_defending.merge(team_names, on='TEAM_FBREF_ID', how='left')
team_attacking = team_attacking.merge(team_names, on='TEAM_FBREF_ID', how='left')

In [225]:
team_attacking = team_attacking.merge(team_standard, on=['TEAM_FBREF_ID','SEASON','TEAM_NAME'], how='left')
team_defending = team_defending.merge(team_standard, on=['TEAM_FBREF_ID','SEASON','TEAM_NAME'], how='left')

In [229]:
team_attacking

,TEAM_FBREF_ID,SEASON,GOALS_SCORED,NPXG,SHOTS,SHOTS_ON_TARGET,PASS_COMPLETED,PASS_ATTEMPTED,TAKEONS_ATTEMPTED,TAKEONS_COMPLETED,CROSSES_INTO_PA,FOULS_AGAINST,TEAM_NAME,MATCHES_PLAYED,Goals/Game,Pass Completion (%),Fouls Against/Game,NPxG/Game,Shots/Game,Shots On Target (%),Take_ons Attempted/Game,Crosses into Pen Area
0,18bb7c10,2223,88,69.16,589,194,18281,21969,751,351,62,435,Arsenal,38,2.315789,0.832127,11.447368,1.82,15.500000,0.329372,19.763158,62
1,8602292d,2223,51,47.12,427,145,13782,17396,681,292,80,498,Aston Villa,38,1.342105,0.792251,13.105263,1.24,11.236842,0.339578,17.921053,80
2,4ba7cbea,2223,37,38.76,358,126,11715,15381,735,321,75,365,Bournemouth,38,0.973684,0.761654,9.605263,1.02,9.421053,0.351955,19.342105,75
3,cd051869,2223,58,50.54,400,147,11317,15551,597,264,87,399,Brentford,38,1.526316,0.727735,10.500000,1.33,10.526316,0.367500,15.710526,87
4,d07537b9,2223,72,68.78,607,219,19210,22888,724,343,78,407,Brighton,38,1.894737,0.839304,10.710526,1.81,15.973684,0.360791,19.052632,78
5,cff3d9bb,2223,38,47.12,481,151,19147,22836,833,375,68,462,Chelsea,38,1.000000,0.838457,12.157895,1.24,12.657895,0.313929,21.921053,68
6,47c64c55,2223,40,36.86,423,133,13395,17308,784,367,51,479,Crystal Palace,38,1.052632,0.773920,12.605263,0.97,11.131579,0.314421,20.631579,51
7,d3fd31cc,2223,34,42.94,426,143,11790,15783,669,305,67,382,Everton,38,0.894737,0.747006,10.052632,1.13,11.210526,0.335681,17.605263,67
8,fd962109,2223,55,39.14,420,137,13974,18014,592,269,97,405,Fulham,38,1.447368,0.775730,10.657895,1.03,11.052632,0.326190,15.578947,97
9,5bfb9659,2223,48,44.84,459,141,12437,16924,688,268,70,405,Leeds United,38,1.263158,0.734874,10.657895,1.18,12.078947,0.307190,18.105263,70


In [227]:
team_attacking['Goals/Game'] = team_attacking['GOALS_SCORED']/team_attacking['MATCHES_PLAYED']
team_attacking['Pass Completion (%)'] = team_attacking['PASS_COMPLETED']/team_attacking['PASS_ATTEMPTED']
team_attacking['Fouls Against/Game'] = team_attacking['FOULS_AGAINST']/team_attacking['MATCHES_PLAYED']
team_attacking['NPxG/Game'] = team_attacking['NPXG']/team_attacking['MATCHES_PLAYED']
team_attacking['Shots/Game'] = team_attacking['SHOTS']/team_attacking['MATCHES_PLAYED']
team_attacking['Shots On Target (%)'] = team_attacking['SHOTS_ON_TARGET']/team_attacking['SHOTS']
team_attacking['Take_ons Attempted/Game'] = team_attacking['TAKEONS_ATTEMPTED']/team_attacking['MATCHES_PLAYED']
team_attacking['Crosses into Pen Area'] = team_attacking['CROSSES_INTO_PA']

In [228]:
team_defending['Clearances/Game'] = team_defending['CLEARANCES']/team_defending['MATCHES_PLAYED']
team_defending['Fouls Made/Game'] = team_defending['FOULS_MADE']/team_defending['MATCHES_PLAYED']
team_defending['Conceded/Game'] = team_defending['GOALS_CONCEDED']/team_defending['MATCHES_PLAYED']
team_defending['xG Against/Game'] = team_defending['XG_AGAINST']/team_defending['MATCHES_PLAYED']
team_defending['Tackles Attempted/Game'] = team_defending['TACKLES']/team_defending['MATCHES_PLAYED']
team_defending['Tackles Won (%)'] = team_defending['TACKLES_WON']/team_defending['TACKLES']
team_defending['Interceptions/Game'] = team_defending['INTERCEPTIONS']/team_defending['MATCHES_PLAYED']
team_defending['Blocked Shots/Game'] = team_defending['BLOCKED_SHOTS']/team_defending['MATCHES_PLAYED']

In [230]:
team_attacking= team_attacking[['SEASON', 'TEAM_NAME', 'Goals/Game', 'Pass Completion (%)', 'Fouls Against/Game', 'NPxG/Game', 
                                'Shots/Game', 'Shots On Target (%)', 'Take_ons Attempted/Game', 'Crosses into Pen Area']]


In [231]:
team_defending= team_defending[['SEASON', 'TEAM_NAME', 'Clearances/Game', 'Fouls Made/Game', 'Conceded/Game', 'xG Against/Game',
                                'Tackles Attempted/Game', 'Tackles Won (%)', 'Interceptions/Game', 'Blocked Shots/Game']]

In [232]:
team_attacking_average = team_attacking.drop(columns=['TEAM_NAME']).groupby('SEASON').mean().reset_index()
team_attacking_average['TEAM_NAME'] = 'Average_2223'
team_attacking = pd.concat([team_attacking, team_attacking_average], ignore_index=True)

In [233]:
team_defending_average = team_defending.drop(columns=['TEAM_NAME']).groupby('SEASON').mean().reset_index()
team_defending_average['TEAM_NAME'] = 'Average_2223'
team_defending = pd.concat([team_defending, team_defending_average], ignore_index=True)

In [237]:
scaler = MinMaxScaler()
team_defending_scaled = (team_defending.drop(['TEAM_NAME','SEASON'], axis=1))
team_defending_scaled_1 = team_defending_scaled.drop(['Fouls Made/Game', 'Conceded/Game', 'xG Against/Game'], axis=1)
team_defending_scaled_2 = team_defending_scaled[['Fouls Made/Game', 'Conceded/Game', 'xG Against/Game']]
team_defending_scaled_1 = pd.DataFrame(scaler.fit_transform(team_defending_scaled_1), columns=team_defending_scaled_1.columns)
team_defending_scaled_1 = pd.DataFrame(scaler.fit_transform(team_defending_scaled_1), columns=team_defending_scaled_1.columns)
scaler = MinMaxScaler()
team_defending_scaled_2 = pd.DataFrame(scaler.fit_transform(team_defending_scaled_2), columns=team_defending_scaled_2.columns)
team_defending_scaled_2 = pd.DataFrame(scaler.fit_transform(1-team_defending_scaled_2), columns=team_defending_scaled_2.columns)
team_defending_scaled_2 = pd.DataFrame(scaler.inverse_transform(team_defending_scaled_2), columns=team_defending_scaled_2.columns)
team_defending_scaled = pd.concat([team_defending_scaled_1, team_defending_scaled_2], axis=1)
team_defending_scaled = pd.concat([team_defending[['TEAM_NAME','SEASON']], team_defending_scaled], axis=1)

In [239]:
scaler = MinMaxScaler()
team_attacking_scaled = (team_attacking.drop(['TEAM_NAME','SEASON'], axis=1))
team_attacking_scaled = pd.DataFrame(scaler.fit_transform(team_attacking_scaled), 
                                     columns=team_attacking_scaled.columns)
team_attacking_scaled = pd.concat([team_attacking[['TEAM_NAME','SEASON']], team_attacking_scaled], axis=1)

In [240]:
team_attacking_radar_1 = team_attacking_scaled.melt(id_vars=["SEASON", "TEAM_NAME"]).sort_values(by=['TEAM_NAME']).rename(columns={'value':'norm_value'})
team_attacking_radar_2 = team_attacking.melt(id_vars=["SEASON", "TEAM_NAME"]).sort_values(by=['TEAM_NAME'])

In [241]:
team_defending_radar_1 = team_defending_scaled.melt(id_vars=["SEASON", "TEAM_NAME"]).sort_values(by=['TEAM_NAME']).rename(columns={'value':'norm_value'})
team_defending_radar_2 = team_defending.melt(id_vars=["SEASON", "TEAM_NAME"]).sort_values(by=['TEAM_NAME'])

In [242]:
team_defending_radar = team_defending_radar_1.merge(team_defending_radar_2, on=['SEASON','TEAM_NAME','variable'], how='left')
team_attacking_radar = team_attacking_radar_1.merge(team_attacking_radar_2, on=['SEASON','TEAM_NAME','variable'], how='left')

In [243]:
team_standard_radar_1 = team_attacking_scaled[['TEAM_NAME','SEASON','Pass Completion (%)','Goals/Game','NPxG/Game',
                                               'Shots/Game','Shots On Target (%)']]
team_standard_radar_2 = team_attacking[['TEAM_NAME','SEASON','Pass Completion (%)','Goals/Game','NPxG/Game',
                                               'Shots/Game','Shots On Target (%)']]
team_standard_radar_3 = team_defending_scaled[['TEAM_NAME','SEASON','Tackles Won (%)', 'Conceded/Game', 'xG Against/Game']]
team_standard_radar_4 = team_defending[['TEAM_NAME','SEASON','Tackles Won (%)', 'Conceded/Game', 'xG Against/Game']]

In [244]:
team_standard_radar_5 = team_standard_radar_1.merge(team_standard_radar_3, on=['TEAM_NAME','SEASON'], how='left')
team_standard_radar_6 = team_standard_radar_2.merge(team_standard_radar_4, on=['TEAM_NAME','SEASON'], how='left')

In [245]:
team_standard_radar_5 = team_standard_radar_5.melt(id_vars=["SEASON", "TEAM_NAME"]).sort_values(by=['TEAM_NAME']).rename(columns={'value':'norm_value'})

In [246]:
team_standard_radar_6 = team_standard_radar_6.melt(id_vars=["SEASON", 
                                                            "TEAM_NAME"]).sort_values(by=['TEAM_NAME'])

In [247]:
team_standard_radar = team_standard_radar_5.merge(team_standard_radar_6, on=['SEASON','TEAM_NAME','variable'], how='left')

In [255]:
team_defending_radar.rename(columns={'variable':'VARIABLE','norm_value':"NORM_VALUE",
                                     'value':'VALUE'}, inplace=True)

In [256]:
team_attacking_radar.rename(columns={'variable':'VARIABLE','norm_value':"NORM_VALUE",
                                     'value':'VALUE'}, inplace=True)

In [257]:
team_standard_radar.rename(columns={'variable':'VARIABLE','norm_value':"NORM_VALUE",
                                     'value':'VALUE'}, inplace=True)

Upload to Snowflake now

In [249]:
SNOWFLAKE_USER = 'kbharaj3'
SNOWFLAKE_PASSWORD = 'Snowfl@key0014'
SNOWFLAKE_ACCOUNT = 'qx25653.ca-central-1.aws'
SNOWFLAKE_WAREHOUSE = 'FOOTY_STORE'
SNOWFLAKE_DATABASE = 'GEGENSTATS'
SNOWFLAKE_SCHEMA = 'RADAR_CHARTS'

In [250]:
conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
    warehouse=SNOWFLAKE_WAREHOUSE,
    database=SNOWFLAKE_DATABASE,
    schema=SNOWFLAKE_SCHEMA
    )

In [251]:
cur = conn.cursor()
cur.execute(f"USE WAREHOUSE {SNOWFLAKE_WAREHOUSE}")

In [258]:
create_schema_sql = f"CREATE SCHEMA IF NOT EXISTS {SNOWFLAKE_SCHEMA}"
cur.execute(create_schema_sql)

In [262]:
create_table_query = """
CREATE TABLE IF NOT EXISTS STANDARD_RADAR (
    SEASON INT,
    TEAM_NAME VARCHAR(255),
    VARIABLE VARCHAR(255),
    NORM_VALUE FLOAT,
    VALUE FLOAT,
    PRIMARY KEY (SEASON, TEAM_NAME, VARIABLE)
);
"""

In [263]:
cur.execute(create_table_query)

In [266]:
def insert_to_snowflake_DB(primary_keys, table_name, dataframe):
    cur.execute(f"SELECT {', '.join(primary_keys)} FROM {table_name}")
    existing_ids = cur.fetchall()
    existing_ids = [id for id in existing_ids]
    dataframe.set_index(primary_keys, inplace=True)
    new_rows = dataframe[~dataframe.index.isin(existing_ids)]
    new_rows.reset_index(inplace=True)
    if not new_rows.empty:
        success, nchunks, nrows, _ = write_pandas(conn, new_rows, table_name)
        print(f"Inserted {nrows} new rows")
    else:
        print("No new rows to insert")

In [267]:
insert_to_snowflake_DB(['SEASON', 'TEAM_NAME', 'VARIABLE'], 'STANDARD_RADAR', team_standard_radar)

Inserted 168 new rows


In [269]:
create_table_query = """
CREATE TABLE IF NOT EXISTS ATTACKING_RADAR (
    SEASON INT,
    TEAM_NAME VARCHAR(255),
    VARIABLE VARCHAR(255),
    NORM_VALUE FLOAT,
    VALUE FLOAT,
    PRIMARY KEY (SEASON, TEAM_NAME, VARIABLE)
);
"""

In [270]:
cur.execute(create_table_query)

In [271]:
insert_to_snowflake_DB(['SEASON', 'TEAM_NAME', 'VARIABLE'], 'ATTACKING_RADAR', team_attacking_radar)

Inserted 168 new rows


In [273]:
create_table_query = """
CREATE TABLE IF NOT EXISTS DEFENDING_RADAR (
    SEASON INT,
    TEAM_NAME VARCHAR(255),
    VARIABLE VARCHAR(255),
    NORM_VALUE FLOAT,
    VALUE FLOAT,
    PRIMARY KEY (SEASON, TEAM_NAME, VARIABLE)
);
"""

In [274]:
cur.execute(create_table_query)

In [275]:
insert_to_snowflake_DB(['SEASON', 'TEAM_NAME', 'VARIABLE'], 'DEFENDING_RADAR', team_defending_radar)

Inserted 168 new rows
